In [84]:
import math
import random
import scipy.stats as sp

x0 = 0.2
n = 100
lmbd = 0.1



x, y = [], []


def get_expected_value_theory():
    return 1/lmbd

def get_expected_value_practice():
    #return (1/n)*sum(y)
    return sp.describe(y).mean

def get_err (theory, practice):
    return (abs(theory-practice)/theory)*100

def get_expected_value_err():
    return get_err(get_expected_value_theory(), get_expected_value_practice())

def get_dispersion_theory():
    return 1/(lmbd**2) 

def get_dispersion_practice():
    squared_deviation = []
    for yi in y:
        squared_deviation.append( abs(yi-get_expected_value_practice()) )    
    return sum(squared_deviation)

def get_dispersion_err():
    return get_err(get_dispersion_theory(), get_dispersion_practice())

def get_distribution_theory(x):
    return 1-math.exp((-1)*lmbd*x)

def get_interval_count_less_x(x):
    count = 0
    for yi in y:
        if (yi < x):
            count += 1
    return count

def get_distribution_practice(x):
    return get_interval_count_less_x(x)/n

def get_distribution_err(x):
    return abs(get_distribution_theory(x)-get_distribution_practice(x))


def get_y (x):
    return (-1)*(math.log(1-x)/lmbd)


for i in range(0,n):
    x.append(random.randrange(0, 100, 1)/100)
    y.append( get_y(x[-1]) )

print(get_expected_value_err())
print(get_dispersion_err())
print(get_distribution_err(x0))


0.4213169966275387
531.845846213451
0.019801326693244747


In [98]:
#import numpy as np

z = []

def get_digits_with_equal_length ():
    m = 1.44*math.log(n) + 1    
    mean_length = sum(y)/m

    for i in range(n):
            z.insert(i, mean_length)
    return z

def get_digit_length (i):
    return sum(z[0:i])

def get_frequency_density (i):
    delta_i = get_digit_length (i+1)-get_digit_length (i)
    return get_digit_length(i)-
    

7.631445067822852 995.7868300337248


[995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,
 995.7868300337248,


In [102]:

lissy = [1,2,3,4,5,6,7,8]
lissy[0:1]


[1]

In [85]:
! git add * 
! git commit -m !
! git push

The file will have its original line endings in your working directory


[main 9e3c3c8] !
 1 file changed, 31 insertions(+), 13 deletions(-)


To https://github.com/russianSlave/queuing_theory_lab1.git
   83145c8..9e3c3c8  main -> main
